In [35]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 11: Plutonian Pebbles ---</h2><p>The ancient civilization on <a href="/2019/day/20">Pluto</a> was known for its ability to manipulate spacetime, and while The Historians explore their infinite corridors, you've noticed a strange set of physics-defying stones.</p>
<p>At first glance, they seem like normal stones: they're arranged in a perfectly <em>straight line</em>, and each stone has a <em>number</em> engraved on it.</p>
<p>The strange part is that every time you <span title="No, they're not statues. Why do you ask?">blink</span>, the stones <em>change</em>.</p>
<p>Sometimes, the number engraved on a stone changes. Other times, a stone might <em>split in two</em>, causing all the other stones to shift over a bit to make room in their perfectly straight line.</p>
<p>As you observe them for a while, you find that the stones have a consistent behavior. Every time you blink, the stones each <em>simultaneously</em> change according to the <em>first applicable rule</em> in this list:</p>
<ul>
<li>If the stone is engraved with the number <code>0</code>, it is replaced by a stone engraved with the number <code>1</code>.</li>
<li>If the stone is engraved with a number that has an <em>even</em> number of digits, it is replaced by <em>two stones</em>. The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: <code>1000</code> would become stones <code>10</code> and <code>0</code>.)</li>
<li>If none of the other rules apply, the stone is replaced by a new stone; the old stone's number <em>multiplied by 2024</em> is engraved on the new stone.</li>
</ul>
<p>No matter how the stones change, their <em>order is preserved</em>, and they stay on their perfectly straight line.</p>
<p>How will the stones evolve if you keep blinking at them? You take a note of the number engraved on each stone in the line (your puzzle input).</p>
<p>If you have an arrangement of five stones engraved with the numbers <code>0 1 10 99 999</code> and you blink once, the stones transform as follows:</p>
<ul>
<li>The first stone, <code>0</code>, becomes a stone marked <code>1</code>.</li>
<li>The second stone, <code>1</code>, is multiplied by 2024 to become <code>2024</code>.</li>
<li>The third stone, <code>10</code>, is split into a stone marked <code>1</code> followed by a stone marked <code>0</code>.</li>
<li>The fourth stone, <code>99</code>, is split into two stones marked <code>9</code>.</li>
<li>The fifth stone, <code>999</code>, is replaced by a stone marked <code>2021976</code>.</li>
</ul>
<p>So, after blinking once, your five stones would become an arrangement of seven stones engraved with the numbers <code>1 2024 1 0 9 9 2021976</code>.</p>
<p>Here is a longer example:</p>
<pre><code>Initial arrangement:
125 17

After 1 blink:
253000 1 7

After 2 blinks:
253 0 2024 14168

After 3 blinks:
512072 1 20 24 28676032

After 4 blinks:
512 72 2024 2 0 2 4 2867 6032

After 5 blinks:
1036288 7 2 20 24 4048 1 4048 8096 28 67 60 32

After 6 blinks:
2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2
</code></pre>

<p>In this example, after blinking six times, you would have <code>22</code> stones. After blinking 25 times, you would have <code><em>55312</em></code> stones!</p>
<p>Consider the arrangement of stones in front of you. <em>How many stones will you have after blinking 25 times?</em></p>
</article>


In [42]:
from collections import deque
from functools import cache


tests = [
    {
        "name": "Small Example",
        "s": "0 1 10 99 999",
        "times": 1,
        "expected": 7,
    },
    {
        "name": "Example",
        "s": "125 17",
        "times": 25,
        "expected": 55312,
    },
]


def nr_stones_after_blining_slow(s: str, times: int, debug=False) -> int:
    stones = deque([int(st) for st in s.strip().split()])
    if debug:
        print("Initial arrangement:")
        print(" ".join(f"{st}" for st in stones))

    for bl in range(times):
        for _ in range(len(stones)):
            stone = stones.popleft()
            if stone == 0:
                # If the stone is engraved with the number 0,
                # it is replaced by a stone engraved with the number 1.
                stones.append(1)
            elif (l := len(str_stone := str(stone))) & 1 == 0:
                # If the stone is engraved with a number that has an even number of digits,
                # it is replaced by two stones. The left half of the digits are engraved on the new left stone,
                # and the right half of the digits are engraved on the new right stone.
                stones.append(int(str_stone[: l // 2]))
                stones.append(int(str_stone[l // 2 :]))
            else:
                # If none of the other rules apply, the stone is replaced by a new stone;
                # the old stone's number multiplied by 2024 is engraved on the new stone.
                stones.append(2024 * stone)

        if debug:
            print(f"After {bl+1} blink{'s' if bl else ''}")
            print(" ".join(f"{st}" for st in stones))

    return len(stones)


def nr_stones_after_blining(s: str, times: int) -> int:
    @cache
    def update_stone(stone: int, times: int) -> int:
        if times == 0:
            return 1

        if stone == 0:
            # If the stone is engraved with the number 0,
            # it is replaced by a stone engraved with the number 1.
            return update_stone(1, times - 1)
        if (l := len(str_stone := str(stone))) & 1 == 0:
            # If the stone is engraved with a number that has an even number of digits,
            # it is replaced by two stones. The left half of the digits are engraved on the new left stone,
            # and the right half of the digits are engraved on the new right stone.
            return update_stone(int(str_stone[: l // 2]), times - 1) + update_stone(
                int(str_stone[l // 2 :]), times - 1
            )

        # If none of the other rules apply, the stone is replaced by a new stone;
        # the old stone's number multiplied by 2024 is engraved on the new stone.
        return update_stone(2024 * stone, times - 1)

    stones = deque([int(st) for st in s.strip().split()])
    return sum(update_stone(stone, times) for stone in stones)


@test(tests=tests[:])
def partI_test(s: str, times: int) -> int:
    return nr_stones_after_blining(s, times)


Test Small Example passed, for partI_test.
Test Example passed, for partI_test.
Success


In [43]:
with open("../input/day11.txt") as f:
    puzzle = f.read()

print(f"Part I: {nr_stones_after_blining(puzzle, 25)}")

Part I: 218079


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>218079</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Historians sure are taking a long time. To be fair, the infinite corridors <em>are</em> very large.</p>
<p><em>How many stones would you have after blinking a total of 75 times?</em></p>
</article>


In [44]:
print(f"Part II: {nr_stones_after_blining(puzzle, 75)}")

Part II: 259755538429618


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>259755538429618</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
